In [ ]:
import sys
sys.path.append("..") # this adds to path parent directory in order to import utils file
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import random
from tqdm import tqdm
import numpy as np
from sklearn import metrics
from IPython.display import clear_output, Image, display
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.mixture import GaussianMixture
from sklearn.metrics.cluster import adjusted_rand_score

from sklearn.decomposition import PCA
import hyperopt
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample
from sklearn.neighbors import kneighbors_graph
import igraph as ig
import louvain
import time
import pickle
import umap
import os
## Import custom utils
import gmm_utils
import hyperopt_utils

In [ ]:
%load_ext autoreload
import importlib
importlib.reload(gmm_utils)
importlib.reload(hyperopt_utils)

## Hyperopt utils

In [87]:
dataset = 'brainCIDR'
df, truth = gmm_utils.loadData(dataset)

print(df.shape, truth.shape, truth.clusters.unique())
params={}
params['dataset'] = 'brainCIDR'
params['minCellsPerGene'] = 0
params['minGeneDispersion'] = 0
params['log'] = True # True, False
params['scaler'] = 'standardScaleCells'# 
params['pca_comp'] = 10 #range (3, 300)
params['doUmap'] = True #range (3, 300)
params['umap_comp'] = 3
params['nb_neighbors'] =10 #3 -15
xx, clusters = gmm_utils.runLouvain(params)

(420, 22085) (420, 2) [3 4 1 6 2 7 5 0]


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


In [88]:
xx

{'_adjusted_mutual_info_score': 0.5625771697388295,
 '_calinski_harabaz_score': 15.945366017935102,
 '_davies_bouldin_score': 4.183801501646325,
 '_fowlkes_mallows_score': 0.5080219740792018,
 '_homogeneity_score': 0.928458247834661,
 '_purity_score': 0.9619047619047619,
 '_rand_index': 0.36855193542298037,
 '_silhouette_score': -0.061762144919204676,
 'dataset': 'brainCIDR',
 'doUmap': True,
 'log': True,
 'minCellsPerGene': 0,
 'minGeneDispersion': 0,
 'nb_neighbors': 10,
 'pca_comp': 10,
 'scaler': 'standardScaleCells',
 'umap_comp': 3}

,silhouette_score,calinski_harabaz_score,davies_bouldin_score,rand_index,homogeneity_score,purity_score,adjusted_mutual_info_score,fowlkes_mallows_score
0,-0.073198,15.395073,3.981379,0.382285,0.932827,0.959524,0.558982,0.520804


In [118]:

def runHyperopt(trialsFile, resultsFile, space, max_evals = 2, restart = False):
    # Define function to optimise
    def evaluateLouvain(args):
        try:
            resultDict, _ = gmm_utils.runLouvain(args)
            if os.path.isfile(resultsFile):
                results = pd.read_pickle(resultsFile)

                newDf = pd.DataFrame.from_dict(resultDict, orient='index').T
                results = results.append(newDf)

            else:

                results = pd.DataFrame.from_dict(resultDict, orient='index').T
            results.to_pickle(resultsFile)
        except:
            return { 'status' : hyperopt.STATUS_FAIL}

        print(f'>> Result: {resultDict["_rand_index"]}')
        ret = {
            'loss' : -resultDict['_rand_index']
            ,'status' : STATUS_OK
            ,'eval_time' : time.time()        
        }
        return ret

    trials = hyperopt_utils.getTrials(trialsFile ,restart = restart )
    evals_per_epoch = 10
    for e in range(len(trials), max_evals, evals_per_epoch):
        best = fmin(evaluateLouvain
                    ,space
                    ,algo=tpe.suggest 
                    ,max_evals= e + evals_per_epoch
                    ,trials=trials)
        print('Index ', e)
        pickle.dump(trialsFile, open(filename, 'wb'))
    return trials

## Brain CIDR

In [114]:
dataset = 'brainCIDR'
space = {
    'dataset' : dataset
    ,'minCellsPerGene':scope.int(hp.quniform('minCellsPerGene', 0, 5, 1))
    ,'minGeneDispersion':hp.uniform('minGeneDispersion', 0, 1.5)
    ,'log' : hp.choice('log', [True,False])
    ,'scaler' : hp.choice('scaler',
            ['none','standardScaleGenes', 'standardScaleCells', 'robustScaleGenes', 'robustScaleCells'])
    ,'pca_comp' : scope.int(hp.quniform('pca_comp', 5, 300,1))
    ,'doUmap' : hp.choice('doUmap', [True,False])
    ,'umap_comp' : scope.int(hp.quniform('umap_comp', 2, 5,1))
    ,'nb_neighbors' : scope.int(hp.quniform('nb_neighbors', 6, 30, 1))
}

trialsFile = f'{dataset}_louvain_trials.pkl'
resultsFile = f'{dataset}_louvain_results.pkl'

In [115]:
trials=hyperopt_utils.getTrials(trialsFile ,restart = False )

Reload trials size :28


In [119]:
trials = runHyperopt(trialsFile, resultsFile, space, max_evals = 500, restart = True);

Creating new trials...


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


>> Result: 0.45949023595819266


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


>> Result: 0.6919122225619411


...

>> Result: 0.8562652571961182


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


>> Result: 0.5152476967043316


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


>> Result: 0.8658320245210758
Index  490


In [120]:
resultsDf = pd.read_pickle(resultsFile)
resultsDf.shape

(500, 17)

In [121]:
# gmm_utils.plotBestPrediction(summaryDf, dataset)

## PancreaticIsletCIDR

In [123]:
dataset= 'pancreaticIsletCIDR'
df, truth = gmm_utils.loadData(dataset)
print(df.shape, truth.shape, truth.clusters.unique())

(60, 180253) (60, 2) [1 3 5 4 0 2]


In [124]:

space = {
    'dataset' : dataset
    ,'minCellsPerGene':scope.int(hp.quniform('minCellsPerGene', 0, 5, 1))
    ,'minGeneDispersion':hp.uniform('minGeneDispersion', 0, 1.5)
    ,'log' : hp.choice('log', [True,False])
    ,'scaler' : hp.choice('scaler',
            ['none','standardScaleGenes', 'standardScaleCells', 'robustScaleGenes', 'robustScaleCells'])
    ,'pca_comp' : scope.int(hp.quniform('pca_comp', 5, 300,1))
    ,'doUmap' : hp.choice('doUmap', [True,False])
    ,'umap_comp' : scope.int(hp.quniform('umap_comp', 2, 5,1))
    ,'nb_neighbors' : scope.int(hp.quniform('nb_neighbors', 6, 30, 1))
}

trialsFile = f'{dataset}_louvain_trials.pkl'
resultsFile = f'{dataset}_louvain_results.pkl'

In [125]:
trials=hyperopt_utils.getTrials(filename ,restart = False )

Reload trials size :28


In [126]:
trials = runHyperopt(trialsFile, resultsFile, space, max_evals = 500, restart = True);

Creating new trials...


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:

>> Result: 0.5097116821961248
Index  0


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


>> Result: 0.09800134414298772


...

/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


>> Result: 0.34360261091352384


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


>> Result: 0.9678140854290546
Index  490


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [ ]:

gmm_utils.plotBestPrediction(summaryDf, dataset, pca_comp = 4)

## Deng

In [128]:
dataset= 'deng'
df, truth = gmm_utils.loadData(dataset)
print(df.shape, truth.shape, truth.clusters.unique())

(268, 22431) (268, 2) [0 1 2 3 4 5 6 7 8 9]


In [129]:

space = {
    'dataset' : dataset
    ,'minCellsPerGene':scope.int(hp.quniform('minCellsPerGene', 0, 5, 1))
    ,'minGeneDispersion':hp.uniform('minGeneDispersion', 0, 1.5)
    ,'log' : hp.choice('log', [True,False])
    ,'scaler' : hp.choice('scaler',
            ['none','standardScaleGenes', 'standardScaleCells', 'robustScaleGenes', 'robustScaleCells'])
    ,'pca_comp' : scope.int(hp.quniform('pca_comp', 5, 300,1))
    ,'doUmap' : hp.choice('doUmap', [True,False])
    ,'umap_comp' : scope.int(hp.quniform('umap_comp', 2, 5,1))
    ,'nb_neighbors' : scope.int(hp.quniform('nb_neighbors', 6, 30, 1))
}
trialsFile = f'{dataset}_louvain_trials.pkl'
resultsFile = f'{dataset}_louvain_results.pkl'

In [130]:
trials=hyperopt_utils.getTrials(filename ,restart = False )

Reload trials size :38


In [ ]:
trials = runHyperopt(trialsFile, resultsFile, space, max_evals = 500, restart = True);

Creating new trials...


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


>> Result: 0.31421277892895105


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype int64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


>> Result: 0.12802339751000724


...

>> Result: 0.19047103940429633


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


>> Result: 0.3439985476615875


/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/supervised.py:732: FutureWarning: The behavior of AMI will change in version 0.22. To match the behavior of 'v_measure_score', AMI will use average_method='arithmetic' by default.
  FutureWarning)
/opt/anaconda3/envs/astrohack/lib/python3.6/site-packages/sklearn/metrics/cluster/unsupervised.py:342: RuntimeWarning: divide by zero encountered in true_divide
  score = (intra_dists[:, None] + intra_dists) / centroid_distances


>> Result: 0.32897658123396845


In [ ]:
summaryDf = pd.read_pickle(resultsFile)
gmm_utils.plotBestPrediction(summaryDf, dataset, pca_comp = 4)

In [ ]:
bestBic, bestAic, bestSil = optimalNbClustersGMM(pc, params['min_clusters'], params['max_clusters'], 2)

In [ ]:
bestBic, bestAic, bestSil

In [ ]:
for n_clust in n_clusters:
    model = GaussianMixture(n_clust, covariance_type ='full', random_state = 0).fit(pc)
    clusters = model.predict(pc)
    score = adjusted_rand_score(truth.clusters.tolist(), clusters)
    print(f"For {n_clust} clusters, score : {score}")

## Evaluation

In [ ]:
model = GaussianMixture(8, covariance_type ='full', random_state = 0).fit(pc)
clusters = model.predict(pc)
score = adjusted_rand_score(truth.clusters.tolist(), clusters)
score

In [ ]:
truth.clusters.value_counts().shape

In [ ]:
dataset = 'brainCIDR'
df, truth = gmm_utils.loadData(dataset)
umap2D = gmm_utils.getUmap(df, pca_comp = 10)
print(df.shape, truth.shape, truth.clusters.unique())
params={}
params['dataset'] = 'brainCIDR'
params['minCellsPerGene'] = 0
params['minGeneDispersion'] = 0
params['log'] = True # True, False
params['scaler'] = 'standardScaleCells'# 
params['pca_comp'] = 10 #range (3, 300)
params['nb_clusters'] =8 #3 -15
gmm_utils.run(params);